# 0. Web APIを用いた目次情報の取得

　[文化庁メディア芸術データベース　マンガ分野　WebAPI](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)を用いて，分析に必要なデータを入手します．なお，python3を使ったweb APIの利用については，[Python3でjsonを返却するwebAPIにアクセスして結果を出力するまで](http://qiita.com/sakura1116/items/4a11a9f7db9f535397fa)を参考にさせて頂きました．

## 環境構築

```bash
conda env create -f env.yml
```

## 準備

In [1]:
import json
import urllib.request
from time import sleep

## 雑誌巻号検索結果の取得

　以下の関数`search_magazine()`を使って，週刊少年ジャンプの雑誌巻号情報を検索し，結果を変数`magazines`に保存します．ここで取得したユニークIDは，次節の「雑誌巻号情報の取得」に必要になります．

In [3]:
def search_magazine(key='JUMPrgl', n_pages=25):
    """
    「ユニークID」「雑誌巻号ID」あるいは「雑誌コード」にkey含む雑誌を，
    n_pages分取得する関数です．
    """
    
    url = 'https://mediaarts-db.bunka.go.jp/mg/api/v1/results_magazines?id=' + \
        key + '&page='
    magazines = []
    
    for i in range(1, n_pages):
        response = urllib.request.urlopen(url + str(i))
        content = json.loads(response.read().decode('utf8'))
        magazines.extend(content['results'])
    return magazines

　Web APIでは，パラメータ`id`で「ユニークID」「雑誌巻号ID」あるいは「雑誌コード」を，`page`で検索結果の取得ページ番号（1ページあたり100件，デフォルトは1）を指定することができます．ここで，週刊少年ジャンプは「雑誌巻号ID」に`JUMPrgl`を含むため，`id=JUMPrgl`を指定します．また，週刊少年ジャンプの検索結果は合計24ページ（2320件）あるため，`page`に1から24を順次指定する必要があります．
詳細は[WebAPI仕様](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)をご参照ください．

　なお，2017年3月31日より文化庁メディア芸術データベースのURLが変更されたため，[WebAPI仕様](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)に記載のリクエストURL（`https://mediaarts-db.jp/mg/api/v1/results_magazines`）ではなく，新しいURL（`https://mediaarts-db.bunka.go.jp/mg/api/v1/results_magazines`）を使用する必要があることにご注意ください．

In [4]:
magazines = search_magazine()

In [5]:
len(magazines)

2320

In [6]:
magazines[0]

{'date_indication': '1969/11/03',
 'id': 323270,
 'magazine_id': 'JUMPrgl19691103',
 'name': '週刊少年ジャンプ',
 'number_assist': '',
 'number_indication': '20',
 'owner_kawasaki': 0,
 'owner_kikuyo': 0,
 'owner_kyoto': 1,
 'owner_meiji': 0,
 'owner_ndl': 0,
 'owner_osaka': 0,
 'publisher': '集英社',
 'vol_num_totalval': '2巻24号(通巻35)'}

In [7]:
magazines[-1]

{'date_indication': '2016/12/12',
 'id': 475979,
 'magazine_id': 'JUMPrgl20161212',
 'name': '週刊少年ジャンプ',
 'number_assist': '',
 'number_indication': '52',
 'owner_kawasaki': 0,
 'owner_kikuyo': 0,
 'owner_kyoto': 0,
 'owner_meiji': 0,
 'owner_ndl': 1,
 'owner_osaka': 0,
 'publisher': '',
 'vol_num_totalval': '49巻48号(通巻2389)'}

## 雑誌巻号情報の取得

　以下の関数`extract_data()`で必要な目次情報を抽出し，`save_data()`で目次情報を保存します．

In [8]:
def extract_data(content):
    """
    contentに含まれる目次情報を取得する関数です．
    - year: 発行年
    - no: 号数
    - title: 作品名
    - author: 著者
    - color: カラーか否か
    - pages: 掲載ページ数
    - start_page: 作品のスタートページ
    - best: 巻頭から数えた掲載順
    - worst: 巻末から数えた掲載順
    """
    
    # マンガ作品のみ抽出します．
    comics = [comic for comic in content['contents'] 
             if comic['category']=='マンガ作品'] 
    data = []
    year = int(content['basics']['date_indication'][:4])
    
    # 号数が記載されていない場合があるので，例外処理が必要です．
    try:
        no = int(content['basics']['number_indication'])
    except ValueError:
        no = content['basics']['number_indication']
    
    for comic in comics:
        title= comic['work']
        if not title:
            continue
            
        # ページ数が記載されていない作品があるので，例外処理が必要です．
        # 特に理由はないですが，無記載の作品は10ページとして処理を進めます．
        try:
            pages = int(comic['work_pages'])
        except ValueError:
            pages = 10

        # 「いぬまるだしっ」等，1週に複数話掲載されている作品に対応するため
        # data中にすでにtitleが含まれる場合は，新規datumとして登録せずに，
        # 既存のdatumのページ数のみ加算します．
        if len(data) > 0 and title in [datum['title'] for datum in data]:
            data[[datum['title'] for datum in 
                  data].index(title)]['pages'] += pages
        else:
            data.append({
                'year': year,
                'no': no,
                'title': comic['work'],
                'author': comic['author'],
                'subtitle': comic['subtitle'],
                'color': comic['note'].count('カラー'),
                'pages': int(comic['work_pages']),
                'start_pages': int(comic['start_page'])
            })

    # 企画物のミニマンガを除外するため，合計5ページ以下のdatumはリストから除外します．
    filterd_data = [datum for datum in data if datum['pages'] > 5]
    for n, datum in enumerate(filterd_data):
        datum['best'] = n + 1
        datum['worst'] = len(filterd_data) - n
        
    return filterd_data

　泥臭い話ですが，一部のギャグ漫画の扱いに苦労しました．例えば，「[いぬまるだしっ](https://mediaarts-db.bunka.go.jp/mg/magazine_works/545?ids%5B%5D=545)」は，基本的に一週間に2話ずつ掲載していましたが，データベースでは各話が別々の行に記載されています．これらを１つの作品として見なす必要があるので，当該`comic`の`title`が`data`中にある場合は，別`datum`として`data`に追加せず，既存の`datum`の`pages`を加算する処理を行っています．また，例えば「[ピューと吹く!ジャガー](https://mediaarts-db.bunka.go.jp/mg/comic_works/83870)」は，その人気に関係なく（実際めちゃくちゃ面白かったです），連載中は常に雑誌の最後に掲載されていました．これを外れ値として除外するかどうかで悩みましたが，結局残すことにしました．

In [14]:
def save_data(magazines, offset=0, file_name='data/wj-api.json'):
    """
    magazinesに含まれる全てのmagazineについて，先頭からoffset以降の巻号の
    目次情報を取得し，file_nameに保存する関数です．
    """
    
    url = 'https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id='
    
    #　ファイル先頭行
    if offset == 0:
        with open(file_name, 'w') as f:
            f.write('[\n')
        
    with open(file_name, 'a') as f:
        
        # magazines中のmagazine毎にWeb APIを叩きます．
        for m, magazine in enumerate(magazines[offset:]):
            response = urllib.request.urlopen(url + str(magazine['id']),
                                              timeout=30)
            content = json.loads(response.read().decode('utf8'))
            
            # 前記の関数extract_data()で，必要な情報を抽出します．
            comics = extract_data(content)
            print('{0:4d}/{1}: Extracted data from {2}'.\
                  format(m + offset, len(magazines), url + str(magazine['id'])))
            
            # comics中の各comicについて，file_nameに情報を保存します．
            for n, comic in enumerate(comics):
                
                # ファイル先頭以外の，magazineの最初のcomicの場合は，
                # まず',\n'を追記．
                if m + offset > 0 and n == 0:
                    f.write(',\n')
                
                json.dump(comic, f, ensure_ascii=False)
                
                # 最後のcomic以外は',\n'を追記．
                if not n == len(comics) - 1:
                    f.write(',\n')
            print('{0:9}: Saved data to {1}'.format(' ', file_name))
            
            # サーバへの負荷を抑えるため，必ず一時停止します．
            sleep(3)
            
    # ファイル最終行
    with open(file_name, 'a') as f:
        f.write(']')

　タイムアウトに柔軟に対応するため，目次情報を一括処理せず，無理やり逐次処理にしています．また，サーバに負荷をかけないよう，`sleep()`で一時停止していることにご注意ください．
 
　なお，2017年3月31日より文化庁メディア芸術データベースのURLが変更されたため，[WebAPI仕様](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)に記載のリクエストURL（`https://mediaarts-db.jp/mg/api/v1/magazine`）ではなく，新しいURL（`https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine`）を使用する必要があることにご注意ください．
 
 　以下を実行し，`wj-api.json`にデータを保存します．

In [11]:
save_data(magazines)

   0/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323270
         : Saved data to data/wj-api.json
   1/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323269
         : Saved data to data/wj-api.json
   2/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323268
         : Saved data to data/wj-api.json
   3/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323267
         : Saved data to data/wj-api.json
   4/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323266
         : Saved data to data/wj-api.json
   5/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323265
         : Saved data to data/wj-api.json
   6/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323264
         : Saved data to data/wj-api.json
   7/2320: Extracted data from https://mediaarts-db.bun

  61/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323223
         : Saved data to data/wj-api.json
  62/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323222
         : Saved data to data/wj-api.json
  63/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323220
         : Saved data to data/wj-api.json
  64/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323219
         : Saved data to data/wj-api.json
  65/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323218
         : Saved data to data/wj-api.json
  66/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323217
         : Saved data to data/wj-api.json
  67/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323216
         : Saved data to data/wj-api.json
  68/2320: Extracted data from https://mediaarts-db.bun

 122/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323161
         : Saved data to data/wj-api.json
 123/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323160
         : Saved data to data/wj-api.json
 124/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323159
         : Saved data to data/wj-api.json
 125/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323158
         : Saved data to data/wj-api.json
 126/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323157
         : Saved data to data/wj-api.json
 127/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323156
         : Saved data to data/wj-api.json
 128/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323155
         : Saved data to data/wj-api.json
 129/2320: Extracted data from https://mediaarts-db.bun

 183/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323100
         : Saved data to data/wj-api.json
 184/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323099
         : Saved data to data/wj-api.json
 185/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323098
         : Saved data to data/wj-api.json
 186/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323097
         : Saved data to data/wj-api.json
 187/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323096
         : Saved data to data/wj-api.json
 188/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323095
         : Saved data to data/wj-api.json
 189/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323094
         : Saved data to data/wj-api.json
 190/2320: Extracted data from https://mediaarts-db.bun

 244/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323038
         : Saved data to data/wj-api.json
 245/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323037
         : Saved data to data/wj-api.json
 246/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323036
         : Saved data to data/wj-api.json
 247/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323035
         : Saved data to data/wj-api.json
 248/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323034
         : Saved data to data/wj-api.json
 249/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323033
         : Saved data to data/wj-api.json
 250/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=323032
         : Saved data to data/wj-api.json
 251/2320: Extracted data from https://mediaarts-db.bun

 305/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322974
         : Saved data to data/wj-api.json
 306/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396533
         : Saved data to data/wj-api.json
 307/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322973
         : Saved data to data/wj-api.json
 308/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322972
         : Saved data to data/wj-api.json
 309/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322971
         : Saved data to data/wj-api.json
 310/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322970
         : Saved data to data/wj-api.json
 311/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322969
         : Saved data to data/wj-api.json
 312/2320: Extracted data from https://mediaarts-db.bun

 366/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322914
         : Saved data to data/wj-api.json
 367/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322913
         : Saved data to data/wj-api.json
 368/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322912
         : Saved data to data/wj-api.json
 369/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322911
         : Saved data to data/wj-api.json
 370/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322910
         : Saved data to data/wj-api.json
 371/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322909
         : Saved data to data/wj-api.json
 372/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322908
         : Saved data to data/wj-api.json
 373/2320: Extracted data from https://mediaarts-db.bun

 427/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322853
         : Saved data to data/wj-api.json
 428/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322852
         : Saved data to data/wj-api.json
 429/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322851
         : Saved data to data/wj-api.json
 430/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322850
         : Saved data to data/wj-api.json
 431/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322849
         : Saved data to data/wj-api.json
 432/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322848
         : Saved data to data/wj-api.json
 433/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322847
         : Saved data to data/wj-api.json
 434/2320: Extracted data from https://mediaarts-db.bun

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

タイムアウトした場合は，`offset`を利用して再開します．例えば，`447/2320`でタイムアウトした場合は，`save_data(offset=448)`を実行します．

In [12]:
save_data(magazines, offset=448)

 448/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322832
         : Saved data to data/wj-api.json
 449/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322831
         : Saved data to data/wj-api.json
 450/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322830
         : Saved data to data/wj-api.json
 451/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322829
         : Saved data to data/wj-api.json
 452/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322828
         : Saved data to data/wj-api.json
 453/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322827
         : Saved data to data/wj-api.json
 454/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322826
         : Saved data to data/wj-api.json
 455/2320: Extracted data from https://mediaarts-db.bun

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [13]:
save_data(magazines, offset=500)

 500/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322780
         : Saved data to data/wj-api.json
 501/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322779
         : Saved data to data/wj-api.json
 502/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322778
         : Saved data to data/wj-api.json
 503/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322777
         : Saved data to data/wj-api.json
 504/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322776
         : Saved data to data/wj-api.json
 505/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322775
         : Saved data to data/wj-api.json
 506/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322774
         : Saved data to data/wj-api.json
 507/2320: Extracted data from https://mediaarts-db.bun

 561/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322718
         : Saved data to data/wj-api.json
 562/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322717
         : Saved data to data/wj-api.json
 563/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322716
         : Saved data to data/wj-api.json
 564/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322715
         : Saved data to data/wj-api.json
 565/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322714
         : Saved data to data/wj-api.json
 566/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322713
         : Saved data to data/wj-api.json
 567/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322712
         : Saved data to data/wj-api.json
 568/2320: Extracted data from https://mediaarts-db.bun

 622/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322657
         : Saved data to data/wj-api.json
 623/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322656
         : Saved data to data/wj-api.json
 624/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322655
         : Saved data to data/wj-api.json
 625/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322654
         : Saved data to data/wj-api.json
 626/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322653
         : Saved data to data/wj-api.json
 627/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322652
         : Saved data to data/wj-api.json
 628/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322650
         : Saved data to data/wj-api.json
 629/2320: Extracted data from https://mediaarts-db.bun

 683/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322597
         : Saved data to data/wj-api.json
 684/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322596
         : Saved data to data/wj-api.json
 685/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322595
         : Saved data to data/wj-api.json
 686/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322594
         : Saved data to data/wj-api.json
 687/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322593
         : Saved data to data/wj-api.json
 688/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322592
         : Saved data to data/wj-api.json
 689/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322591
         : Saved data to data/wj-api.json
 690/2320: Extracted data from https://mediaarts-db.bun

 744/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322534
         : Saved data to data/wj-api.json
 745/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322533
         : Saved data to data/wj-api.json
 746/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322532
         : Saved data to data/wj-api.json
 747/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322531
         : Saved data to data/wj-api.json
 748/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322530
         : Saved data to data/wj-api.json
 749/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322529
         : Saved data to data/wj-api.json
 750/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322528
         : Saved data to data/wj-api.json
 751/2320: Extracted data from https://mediaarts-db.bun

...

1049/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322223
         : Saved data to data/wj-api.json
1050/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322222
         : Saved data to data/wj-api.json
1051/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322221
         : Saved data to data/wj-api.json
1052/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322220
         : Saved data to data/wj-api.json
1053/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322219
         : Saved data to data/wj-api.json
1054/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322218
         : Saved data to data/wj-api.json
1055/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322217
         : Saved data to data/wj-api.json
1056/2320: Extracted data from https://mediaarts-db.bun

1110/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322161
         : Saved data to data/wj-api.json
1111/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322160
         : Saved data to data/wj-api.json
1112/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322159
         : Saved data to data/wj-api.json
1113/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322158
         : Saved data to data/wj-api.json
1114/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322157
         : Saved data to data/wj-api.json
1115/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322156
         : Saved data to data/wj-api.json
1116/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322155
         : Saved data to data/wj-api.json
1117/2320: Extracted data from https://mediaarts-db.bun

1171/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322099
         : Saved data to data/wj-api.json
1172/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322098
         : Saved data to data/wj-api.json
1173/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322097
         : Saved data to data/wj-api.json
1174/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322096
         : Saved data to data/wj-api.json
1175/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322095
         : Saved data to data/wj-api.json
1176/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322094
         : Saved data to data/wj-api.json
1177/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322093
         : Saved data to data/wj-api.json
1178/2320: Extracted data from https://mediaarts-db.bun

1232/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322033
         : Saved data to data/wj-api.json
1233/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322032
         : Saved data to data/wj-api.json
1234/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322031
         : Saved data to data/wj-api.json
1235/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322030
         : Saved data to data/wj-api.json
1236/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322029
         : Saved data to data/wj-api.json
1237/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322028
         : Saved data to data/wj-api.json
1238/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=322027
         : Saved data to data/wj-api.json
1239/2320: Extracted data from https://mediaarts-db.bun

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [10]:
save_data(magazines, offset=1269)

1269/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321996
         : Saved data to data/wj-api.json
1270/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321995
         : Saved data to data/wj-api.json
1271/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321994
         : Saved data to data/wj-api.json
1272/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321993
         : Saved data to data/wj-api.json
1273/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321992
         : Saved data to data/wj-api.json
1274/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321991
         : Saved data to data/wj-api.json
1275/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321990
         : Saved data to data/wj-api.json
1276/2320: Extracted data from https://mediaarts-db.bun

1330/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321935
         : Saved data to data/wj-api.json
1331/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321934
         : Saved data to data/wj-api.json
1332/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321932
         : Saved data to data/wj-api.json
1333/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321931
         : Saved data to data/wj-api.json
1334/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321930
         : Saved data to data/wj-api.json
1335/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321929
         : Saved data to data/wj-api.json
1336/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321928
         : Saved data to data/wj-api.json
1337/2320: Extracted data from https://mediaarts-db.bun

1391/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321873
         : Saved data to data/wj-api.json
1392/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321872
         : Saved data to data/wj-api.json
1393/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321871
         : Saved data to data/wj-api.json
1394/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321870
         : Saved data to data/wj-api.json
1395/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321869
         : Saved data to data/wj-api.json
1396/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321868
         : Saved data to data/wj-api.json
1397/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321867
         : Saved data to data/wj-api.json
1398/2320: Extracted data from https://mediaarts-db.bun

1452/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321812
         : Saved data to data/wj-api.json
1453/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321811
         : Saved data to data/wj-api.json
1454/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321810
         : Saved data to data/wj-api.json
1455/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321809
         : Saved data to data/wj-api.json
1456/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321808
         : Saved data to data/wj-api.json
1457/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321807
         : Saved data to data/wj-api.json
1458/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321806
         : Saved data to data/wj-api.json
1459/2320: Extracted data from https://mediaarts-db.bun

1513/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321752
         : Saved data to data/wj-api.json
1514/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321751
         : Saved data to data/wj-api.json
1515/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321750
         : Saved data to data/wj-api.json
1516/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321749
         : Saved data to data/wj-api.json
1517/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321748
         : Saved data to data/wj-api.json
1518/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321747
         : Saved data to data/wj-api.json
1519/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321746
         : Saved data to data/wj-api.json
1520/2320: Extracted data from https://mediaarts-db.bun

...

1696/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321565
         : Saved data to data/wj-api.json
1697/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321563
         : Saved data to data/wj-api.json
1698/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321562
         : Saved data to data/wj-api.json
1699/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321561
         : Saved data to data/wj-api.json
1700/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321560
         : Saved data to data/wj-api.json
1701/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321559
         : Saved data to data/wj-api.json
1702/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321558
         : Saved data to data/wj-api.json
1703/2320: Extracted data from https://mediaarts-db.bun

1757/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321501
         : Saved data to data/wj-api.json
1758/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321500
         : Saved data to data/wj-api.json
1759/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321499
         : Saved data to data/wj-api.json
1760/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321498
         : Saved data to data/wj-api.json
1761/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321497
         : Saved data to data/wj-api.json
1762/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321496
         : Saved data to data/wj-api.json
1763/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321495
         : Saved data to data/wj-api.json
1764/2320: Extracted data from https://mediaarts-db.bun

1818/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321440
         : Saved data to data/wj-api.json
1819/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321439
         : Saved data to data/wj-api.json
1820/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321438
         : Saved data to data/wj-api.json
1821/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321437
         : Saved data to data/wj-api.json
1822/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321436
         : Saved data to data/wj-api.json
1823/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321435
         : Saved data to data/wj-api.json
1824/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321434
         : Saved data to data/wj-api.json
1825/2320: Extracted data from https://mediaarts-db.bun

1879/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321379
         : Saved data to data/wj-api.json
1880/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321378
         : Saved data to data/wj-api.json
1881/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321377
         : Saved data to data/wj-api.json
1882/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321376
         : Saved data to data/wj-api.json
1883/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321375
         : Saved data to data/wj-api.json
1884/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321374
         : Saved data to data/wj-api.json
1885/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321373
         : Saved data to data/wj-api.json
1886/2320: Extracted data from https://mediaarts-db.bun

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [11]:
save_data(magazines, offset=1889)

1889/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321369
         : Saved data to data/wj-api.json
1890/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321368
         : Saved data to data/wj-api.json
1891/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321367
         : Saved data to data/wj-api.json
1892/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321366
         : Saved data to data/wj-api.json
1893/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321365
         : Saved data to data/wj-api.json
1894/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321364
         : Saved data to data/wj-api.json
1895/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321363
         : Saved data to data/wj-api.json
1896/2320: Extracted data from https://mediaarts-db.bun

1950/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321308
         : Saved data to data/wj-api.json
1951/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321307
         : Saved data to data/wj-api.json
1952/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321306
         : Saved data to data/wj-api.json
1953/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321305
         : Saved data to data/wj-api.json
1954/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321304
         : Saved data to data/wj-api.json
1955/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321303
         : Saved data to data/wj-api.json
1956/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321302
         : Saved data to data/wj-api.json
1957/2320: Extracted data from https://mediaarts-db.bun

2011/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321249
         : Saved data to data/wj-api.json
2012/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321248
         : Saved data to data/wj-api.json
2013/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321247
         : Saved data to data/wj-api.json
2014/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321246
         : Saved data to data/wj-api.json
2015/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321245
         : Saved data to data/wj-api.json
2016/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321244
         : Saved data to data/wj-api.json
2017/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=321243
         : Saved data to data/wj-api.json
2018/2320: Extracted data from https://mediaarts-db.bun

2072/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396565
         : Saved data to data/wj-api.json
2073/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396566
         : Saved data to data/wj-api.json
2074/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396567
         : Saved data to data/wj-api.json
2075/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396568
         : Saved data to data/wj-api.json
2076/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396569
         : Saved data to data/wj-api.json
2077/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396570
         : Saved data to data/wj-api.json
2078/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396571
         : Saved data to data/wj-api.json
2079/2320: Extracted data from https://mediaarts-db.bun

2133/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396626
         : Saved data to data/wj-api.json
2134/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396627
         : Saved data to data/wj-api.json
2135/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396628
         : Saved data to data/wj-api.json
2136/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396629
         : Saved data to data/wj-api.json
2137/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396630
         : Saved data to data/wj-api.json
2138/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396631
         : Saved data to data/wj-api.json
2139/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396632
         : Saved data to data/wj-api.json
2140/2320: Extracted data from https://mediaarts-db.bun

2194/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396687
         : Saved data to data/wj-api.json
2195/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396688
         : Saved data to data/wj-api.json
2196/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396689
         : Saved data to data/wj-api.json
2197/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396690
         : Saved data to data/wj-api.json
2198/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396691
         : Saved data to data/wj-api.json
2199/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396692
         : Saved data to data/wj-api.json
2200/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=396693
         : Saved data to data/wj-api.json
2201/2320: Extracted data from https://mediaarts-db.bun

2255/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=458214
         : Saved data to data/wj-api.json
2256/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=458215
         : Saved data to data/wj-api.json
2257/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=458216
         : Saved data to data/wj-api.json
2258/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=458217
         : Saved data to data/wj-api.json
2259/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=458218
         : Saved data to data/wj-api.json
2260/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=458219
         : Saved data to data/wj-api.json
2261/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=458220
         : Saved data to data/wj-api.json
2262/2320: Extracted data from https://mediaarts-db.bun

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [12]:
save_data(magazines, offset=2274)

2274/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=471713
         : Saved data to data/wj-api.json
2275/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=471714
         : Saved data to data/wj-api.json
2276/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=471715
         : Saved data to data/wj-api.json
2277/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=471716
         : Saved data to data/wj-api.json
2278/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=471717
         : Saved data to data/wj-api.json
2279/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=471718
         : Saved data to data/wj-api.json
2280/2320: Extracted data from https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id=471719
         : Saved data to data/wj-api.json
2281/2320: Extracted data from https://mediaarts-db.bun